<a href="https://colab.research.google.com/github/LucaMorocco/Luca-s-room/blob/main/Coding_School_data_science_project_Baseball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score

Functions and Libraries import

In [5]:
from google.colab import files
uploaded = files.upload()

#Base ball data set from caggle upload

Saving 500hits.csv to 500hits.csv


In [6]:
df_one = pd.read_csv('/content/500hits.csv',encoding = 'latin-1')
df_one.head()

,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA,HOF
0,Ty Cobb,24,3035,11434,2246,4189,724,295,117,726,1249,357,892,178,0.366,1
1,Stan Musial,22,3026,10972,1949,3630,725,177,475,1951,1599,696,78,31,0.331,1
2,Tris Speaker,22,2789,10195,1882,3514,792,222,117,724,1381,220,432,129,0.345,1
3,Derek Jeter,20,2747,11195,1923,3465,544,66,260,1311,1082,1840,358,97,0.310,1
4,Honus Wagner,21,2792,10430,1736,3430,640,252,101,0,963,327,722,15,0.329,1


In [7]:
df_one.info()
#It seems like there are no missing values,
#but there are probably incorrect values
# I saw a zero in RBIs, so definitly need to find zeros.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PLAYER  465 non-null    object 
 1   YRS     465 non-null    int64  
 2   G       465 non-null    int64  
 3   AB      465 non-null    int64  
 4   R       465 non-null    int64  
 5   H       465 non-null    int64  
 6   2B      465 non-null    int64  
 7   3B      465 non-null    int64  
 8   HR      465 non-null    int64  
 9   RBI     465 non-null    int64  
 10  BB      465 non-null    int64  
 11  SO      465 non-null    int64  
 12  SB      465 non-null    int64  
 13  CS      465 non-null    int64  
 14  BA      465 non-null    float64
 15  HOF     465 non-null    int64  
dtypes: float64(1), int64(14), object(1)
memory usage: 58.3+ KB


In [14]:
df_one.describe()
#There are no missing values, but there are some zeros in the RBI category and the strikeout category that need to be fixed.

,Years Played,Games Played,At Bats,Runs,Hits,Double,Triple,Home Run,Runs Batted In,Walks,Strike Outs,Stolen Bases,Caught Stolen,Batting Average,Hall of Fame
count,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.00000,465.000000,465.000000,465.000000,465.000000,465.000000
mean,17.049462,2048.698925,7511.455914,1150.313978,2170.247312,380.952688,78.554839,201.049462,894.260215,783.56129,847.470968,195.905376,58.083871,0.288712,0.329032
std,2.765186,354.391805,1294.065992,289.635071,424.190773,96.483460,49.363030,143.622664,486.193456,327.43195,489.224289,181.845543,48.027509,0.021208,0.474928
min,11.000000,1331.000000,4981.000000,601.000000,1660.000000,177.000000,3.000000,9.000000,0.000000,239.00000,0.000000,7.000000,0.000000,0.246000,0.000000
25%,15.000000,1802.000000,6523.000000,936.000000,1838.000000,312.000000,41.000000,79.000000,640.000000,535.00000,436.000000,63.000000,22.000000,0.273000,0.000000
50%,17.000000,1993.000000,7241.000000,1104.000000,2076.000000,366.000000,67.000000,178.000000,968.000000,736.00000,825.000000,137.000000,52.000000,0.287000,0.000000
75%,19.000000,2247.000000,8180.000000,1296.000000,2375.000000,436.000000,107.000000,292.000000,1206.000000,955.00000,1226.000000,285.000000,84.000000,0.300000,1.000000
max,26.000000,3308.000000,12364.000000,2295.000000,4189.000000,792.000000,309.000000,755.000000,2297.000000,2190.00000,2597.000000,1406.000000,335.000000,0.366000,2.000000


In [8]:
df_one.columns

Index(['PLAYER', 'YRS', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB',
       'SO', 'SB', 'CS', 'BA', 'HOF'],
      dtype='object')

In [11]:
rename={"PLAYER":"Player","YRS":"Years Played", "G":"Games Played",
        "AB":"At Bats","R":"Runs","H":"Hits","2B":"Doubles","3B":"Triples",
        "HR":"Home Runs","RBI":"Runs Batted In","BB":"Walks"
        ,"SO":"Strike Outs","SB":"Stolen Bases","CS":"Caught Stolen",
        "BA":"Batting Average","HOF":"Hall of Fame"}
df_one = df_one.rename(columns= rename)
df_one.head()

,Player,Years Played,Games Played,At Bats,Runs,Hits,Double,Triple,Home Run,Runs Batted In,Walks,Strike Outs,Stolen Bases,Caught Stolen,Batting Average,Hall of Fame
0,Ty Cobb,24,3035,11434,2246,4189,724,295,117,726,1249,357,892,178,0.366,1
1,Stan Musial,22,3026,10972,1949,3630,725,177,475,1951,1599,696,78,31,0.331,1
2,Tris Speaker,22,2789,10195,1882,3514,792,222,117,724,1381,220,432,129,0.345,1
3,Derek Jeter,20,2747,11195,1923,3465,544,66,260,1311,1082,1840,358,97,0.310,1
4,Honus Wagner,21,2792,10430,1736,3430,640,252,101,0,963,327,722,15,0.329,1


In [16]:
# I am going to try to start finding the missing data and replacing it.
df_one[(df_one['Runs Batted In']<100)]
#looks like Runs batted in category is unusable unless
#I manually replace all the values or drop the rows with zeros.


,Player,Years Played,Games Played,At Bats,Runs,Hits,Double,Triple,Home Run,Runs Batted In,Walks,Strike Outs,Stolen Bases,Caught Stolen,Batting Average,Hall of Fame
4,Honus Wagner,21,2792,10430,1736,3430,640,252,101,0,963,327,722,15,0.329,1
10,Nap Lajoie,21,2480,9589,1504,3242,657,163,83,0,516,85,380,21,0.338,1
23,Cap Anson,22,2276,9101,1719,2995,528,124,97,0,952,294,247,0,0.329,1
25,Sam Crawford,19,2517,9570,1391,2961,458,309,97,0,760,104,366,30,0.309,1
27,Willie Keeler,19,2123,8591,1719,2932,241,145,33,0,524,36,495,0,0.341,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Steve Brodie,12,1437,5699,886,1726,191,89,25,0,420,148,289,0,0.303,0
422,Cy Seymour,16,1528,5682,737,1723,229,96,52,0,354,32,222,0,0.303,0
424,Cupid Childs,13,1456,5618,1214,1720,205,100,20,0,991,117,269,0,0.306,0
448,Hardy Richardson,14,1331,5642,1120,1688,303,126,70,0,377,445,205,0,0.299,0


In [19]:
#This will be for strikeouts
df_one[(df_one['Strike Outs']<115)]
# Strikeouts cannot be use either because the data is incomplete for players
# that played in the early 20th or late 19th century
#Joe Sewell had the lowest strike out percentage so its safe to say
#that most of these numbers are incomplete.

,Player,Years Played,Games Played,At Bats,Runs,Hits,Double,Triple,Home Run,Runs Batted In,Walks,Strike Outs,Stolen Bases,Caught Stolen,Batting Average,Hall of Fame
10,Nap Lajoie,21,2480,9589,1504,3242,657,163,83,0,516,85,380,21,0.338,1
25,Sam Crawford,19,2517,9570,1391,2961,458,309,97,0,760,104,366,30,0.309,1
27,Willie Keeler,19,2123,8591,1719,2932,241,145,33,0,524,36,495,0,0.341,1
91,Bobby Wallace,25,2383,8618,1057,2309,391,143,34,0,774,79,201,2,0.268,1
103,Joe Sewell,14,1903,7132,1141,2226,436,68,49,1055,842,114,74,72,0.312,1
129,Jimmy Collins,14,1725,6795,1055,1999,352,116,65,0,426,32,194,0,0.294,1
143,Elmer Flick,13,1483,5597,950,1752,268,164,48,0,597,0,330,0,0.313,1
159,Lave Cross,21,2275,9072,1333,2645,411,135,47,0,464,90,301,0,0.292,0
206,Fred Tenney,17,1994,7595,1278,2231,270,77,22,0,874,43,285,0,0.294,0
322,Fielder Jones,15,1788,6747,1180,1920,206,75,21,0,817,15,359,0,0.285,0


In [36]:
df_one[(df_one['Caught Stolen']<1)]
#Too many zeros here too so this column must be dropped as well or all players with zeros must be dropped

,Player,Years Played,Games Played,At Bats,Runs,Hits,Double,Triple,Home Run,Runs Batted In,Walks,Strike Outs,Stolen Bases,Caught Stolen,Batting Average,Hall of Fame
12,Paul Waner,20,2549,9459,1627,3152,605,191,113,1309,1091,376,104,0,0.333,1
23,Cap Anson,22,2276,9101,1719,2995,528,124,97,0,952,294,247,0,0.329,1
27,Willie Keeler,19,2123,8591,1719,2932,241,145,33,0,524,36,495,0,0.341,1
28,Jake Beckley,20,2386,9526,1600,2930,473,243,86,0,616,270,315,0,0.308,1
33,Mel Ott,22,2730,9456,1859,2876,488,72,511,1860,1708,896,89,0,0.304,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,Frank McCormick,13,1534,5723,722,1711,334,26,128,951,399,189,27,0,0.299,0
430,Augie Galan,16,1742,5937,1004,1706,336,74,100,830,979,393,123,0,0.287,0
436,Pinky Whitney,12,1539,5765,696,1701,303,56,93,927,400,438,45,0,0.295,0
448,Hardy Richardson,14,1331,5642,1120,1688,303,126,70,0,377,445,205,0,0.299,0


In [39]:
df_one[(df_one['Hall of Fame']> 1)]
# obviously this value is incorrect should be a one



,Player,Years Played,Games Played,At Bats,Runs,Hits,Double,Triple,Home Run,Runs Batted In,Walks,Strike Outs,Stolen Bases,Caught Stolen,Batting Average,Hall of Fame


In [40]:
df_one.iloc[160,15]= 1
df_one.iloc[160,15]
#Fixed that datapoint

np.int64(1)